### Import of required libraries

In [1]:
import os
import pandas as pd
from tqdm.auto import tqdm
from traffic.core import Traffic, Flight
from traffic.data import opensky, navaids, airports
from typing import Union, List, Tuple
from traffic.core.mixins import PointMixin
import os
os.chdir("/home/krum/git/STAR_shortcut_OSN_paper")
from utils import helperfunctions as hf
import plotly.graph_objects as go

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
airport = "LSGG"

### Data fetching

In [3]:
# Paras
folder_daily = f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/daily_new/"
date_range = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# Fetching in daily packages
for date in date_range:
    path = f"{folder_daily}{date.month}/"
    if not os.path.exists(path):
        os.makedirs(path)
    file = path + date.strftime('%Y-%m-%d')+".parquet"
    if not os.path.exists(file):
        print(f"fetching {date}")
        try:
            t = opensky.history(
                start=date.strftime('%Y-%m-%d 00:00'),
                stop=date.strftime('%Y-%m-%d 23:59'),
                bounds=airports[airport].shape.buffer(1.5).bounds,
                # callsign=callsigns,
            )
            t.to_parquet(file)
        except:
            print(f"failed to fetch {date}")

### Processing (clean invalid, assign id, only landings at airport, add landing runway)

In [4]:
# Reduction to landing 04 in monthly packages-----------------------------------
folder_list = os.listdir(folder_daily)
# for each month
for folder in folder_list:
    # List all files in the folder
    file_list = os.listdir(folder_daily + folder)
    filepaths = [os.path.join(folder_daily + folder, file) for file in file_list]

    # Merging to one trafic object
    t = Traffic(
        pd.concat(
            [Traffic.from_file(file).data for file in tqdm(filepaths)],
            ignore_index=True,
        )
    )

    # Cleaning
    t = t.clean_invalid().assign_id().eval(desc="Cleaning", max_workers=20)

    # Reduce to landings at LSGG
    t = (
        t.iterate_lazy()
        .pipe(hf.has_landing_at, "LSGG")
        .eval(desc="landing at LSGG", max_workers=20)
    )

    # Save
    t.to_parquet(
        f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly/landing_{folder}.parquet"
   )

### Merge monthly into one yearly

In [5]:
# Merge all monthly packages to one yearly package------------------------------
file_list = os.listdir(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly/")
file_list = [file for file in file_list if file.startswith("landing_")]
filepaths = [
    os.path.join(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/monthly", file)
    for file in file_list
]
t = Traffic(
    pd.concat(
        [Traffic.from_file(file).data for file in tqdm(filepaths)],
        ignore_index=True,
    )
)
t.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all.parquet")

### Add star info and crop before navaid

In [4]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all.parquet")

belus = navaids.extent("Switzerland")["BELUS"]
kines = navaids.extent("Switzerland")["KINES"]
akito = navaids.extent("Switzerland")["AKITO"]
lusar = navaids["LUSAR"]


def add_star(flight: Flight) -> Union[Flight, None]:
    """
    Add star information to the flight data for LSGG airport.

    Parameters
    ----------
    flight : Flight
        A flight object.

    Returns
    -------
    Flight
        A flight object with star information in an additional column.
    """
    if flight.aligned_on_navpoint(belus, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] == "04":
            flight.data["star"] = "BELU3N"
            flight = hf.crop_before_wp(flight, belus)
            return flight
    
    if flight.aligned_on_navpoint(kines, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] == "04":
            flight.data["star"] = "KINE2N"
            flight = hf.crop_before_wp(flight, kines)
            return flight
    
    if flight.aligned_on_navpoint(akito, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] == "22":
            flight.data["star"] = "AKIT3R"
            flight = hf.crop_before_wp(flight, akito)
            return flight
    
    if flight.aligned_on_navpoint(lusar, 1, "2T", "30s", 20).has():
        if flight.data["rwy"].iloc[0] == "04":
            flight.data["star"] = "LUSA2N"
            flight = hf.crop_before_wp(flight, lusar)
            return flight

# add star information 
t = (
    t.iterate_lazy()
    .pipe(add_star)
    .eval(desc= "add star", max_workers=20)
)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp.parquet"
)

### Remove GAs

In [6]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp.parquet")

# remove GAs
t = (
    t.iterate_lazy()
    .pipe(hf.remove_ga, airport)
    .eval(desc="Removing GAs", max_workers=20)
)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga.parquet"
)

### Crop after runway threshold

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga.parquet")

# remove GAs
t = (
    t.iterate_lazy()
    .pipe(hf.crop_after_th, airport)
    .eval(desc="Cropping at th", max_workers=20)
)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th.parquet"
)

### Filter

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th.parquet")

# Filter
t = t.filter().eval(desc="filtering", max_workers=20)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th_fr.parquet"
)

### Add cumulative distance

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_all_wp_ga_th_fr.parquet")

# Filter
t = t.cumulative_distance().eval(desc="cumulative distance", max_workers=20)

# save
t.to_parquet(
    f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_full.parquet"
)

### Generate dataframe

In [ ]:
# load data
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_full.parquet")

# Initialize an empty list to collect data for each flight
flight_data = []

# Iterate through each flight and collect the relevant information
for flight in tqdm(t):
    id = flight.flight_id
    typecode = flight.typecode
    start = flight.start
    stop = flight.stop
    runway = flight.data.rwy.iloc[0]
    star = flight.data.star.iloc[0]
    distance = flight.data.cumdist.iloc[-1]
    
    # Append the flight data to the list as a tuple or list
    flight_data.append([id, typecode, start, stop, runway, star, distance])

# Create a DataFrame from the collected data
df = pd.DataFrame(flight_data, columns=['id','typecode','start', 'stop', 'runway', 'star', 'distance'])

# Save
df.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_full.parquet")

### Generate subsamples for visualisation

In [15]:
t = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/landing_full.parquet")

sample_2500 = t.sample(2500)
sample_2500.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/sample_2500.parquet")

sample_5000 = t.sample(5000)
sample_5000.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/sample_5000.parquet")

sample_10000 = t.sample(10000)
sample_10000.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/sample_10000.parquet")

belus = t.query('star == "BELU3N"')
belus = belus.sample(min(1000, len(belus)))
belus.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/belu3n.parquet")

kines = t.query('star == "KINE2N"')
kines = kines.sample(min(1000, len(kines)))
kines.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/kine2n.parquet")

akito = t.query('star == "AKIT3R"')
akito = akito.sample(min(1000, len(akito)))
akito.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/akit3r.parquet")

lusar = t.query('star == "LUSA2N"')
lusar = lusar.sample(min(1000, len(lusar)))
lusar.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/{airport}/processed/lusar.parquet")

### Visualisation

In [25]:
# load data
t = Traffic.from_file(f"/home/krum/git/STAR_shortcut_OSN_paper/data/LSGG/lusa2n.parquet")

In [ ]:
# Create a single figure for the map
fig = go.Figure()

for flight in t.sample(50):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="Observed trajectories",
            showlegend=False,
        )
    )

# Add BELUS
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["BELUS"].latitude],
        lon=[navaids["BELUS"].longitude],
        marker=dict(size=10, color="red"),
        text=["BELUS"],
        textposition="bottom left",
        textfont=dict(color="red", size=25),
        name="BELUS",
        showlegend=False,
    )
)

# Add KINES
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["KINES"].latitude],
        lon=[navaids["KINES"].longitude],
        marker=dict(size=10, color="red"),
        text=["KINES"],
        textposition="bottom center",
        textfont=dict(color="red", size=25),
        name="KINES",
        showlegend=False,
    )
)

# Add AKITO
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids.extent("Switzerland")["AKITO"].latitude],
        lon=[navaids.extent("Switzerland")["AKITO"].longitude],
        marker=dict(size=10, color="red"),
        text=["AKITO"],
        textposition="top center",
        textfont=dict(color="red", size=25),
        name="AKITO",
        showlegend=False,
    )
)

# Add LUSAR
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["LUSAR"].latitude],
        lon=[navaids["LUSAR"].longitude],
        marker=dict(size=10, color="red"),
        text=["LUSAR"],
        textposition="top left",
        textfont=dict(color="red", size=25),
        name="LUSAR",
        showlegend=False,
    )
)

# Update layout for the map
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=46.23696078946708,
            lon=6.109110255543277,
        ),
    ),
    legend=dict(
        x=0.65,
        y=0.97,
        traceorder="normal",
        font=dict(size=25),
        bgcolor="rgba(255, 255, 255, 0.7)",
    ),
)

# Show the figure
fig.show()